In [ ]:
# !wget https://course.ccs.neu.edu/cs6220/fall2023/homework-1/netflix-data/movie_titles.csv
# !wget https://course.ccs.neu.edu/cs6220/fall2023/homework-1/netflix-data/combined_data_1.txt
# !wget https://course.ccs.neu.edu/cs6220/fall2023/homework-1/netflix-data/combined_data_2.txt
# !wget https://course.ccs.neu.edu/cs6220/fall2023/homework-1/netflix-data/combined_data_3.txt
# !wget https://course.ccs.neu.edu/cs6220/fall2023/homework-1/netflix-data/combined_data_4.txt
# !wget https://course.ccs.neu.edu/cs6220/fall2023/homework-1/basket_data.csv
# !wget https://course.ccs.neu.edu/cs6220/fall2023/homework-1/netflix-data/probe.txt
# !wget https://course.ccs.neu.edu/cs6220/fall2023/homework-1/netflix-data/qualifying.txt

## Coding Review

In [ ]:
# 1. What is the cardinality in "basket_data.csv"?

import csv

item_set = set()

def cardinality_items(filename):
  '''
  Takes a filename "*.csv" and returns and
  interger
  '''
  with open(filename) as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
      for item in row:
        item_set.add(item.strip())

  return len(item_set)

cardinality_items("/content/basket_data.csv")

21

In [ ]:
# 2. output a list of all possible unique itemsets with non-repeating k items

def all_itemsets(items, k):
    result = [[]]

    for item in items:
        result += [current + [item] for current in result]

    return [comb for comb in result if len(comb) == k]

all_itemsets(["ham", "cheese", "bread"], 2)

[['ham', 'cheese'], ['ham', 'bread'], ['cheese', 'bread']]

## Examing Our First Dataset


In [10]:
# 3. a) How many total records of movie ratings are there in entire dataset

count = 0

for txt in ['combined_data_1.txt', 'combined_data_2.txt', 'combined_data_3.txt', 'combined_data_4.txt']:
  with open(txt) as f:
    for line in f:
      if line.strip().endswith(":"):
        continue
      count += 1

print(count)

100480507


In [ ]:
# 3. b) How many total unique users are there in entire dataset

def find_unique_users(filenames):
  user_set = set()

  for txt in filenames:
    with open(txt, "r") as f:
      for line in f:
        if len(line.split(",")) == 1:
          continue
        user_set.add(line.split(",")[0])

  return len(user_set)

find_unique_users(['combined_data_1.txt', 'combined_data_2.txt', 'combined_data_3.txt', 'combined_data_4.txt'])

480189

In [ ]:
# 3. c) What is the range of years that this data is valid over

def find_range_years(filenames):
  years = set()

  for txt in filenames:
    with open(txt, "r") as f:
      for line in f:
        if len(line.split(',')) == 3:
          year = line.split(",")[2][0:4]
          years.add(int(year))

  return min(list(years)), max(list(years))

min_year, max_year = find_range_years(['combined_data_1.txt', 'combined_data_2.txt', 'combined_data_3.txt', 'combined_data_4.txt'])
print("Range of years: ", min_year, "to", max_year)

Range of years:  1999 to 2005


In [6]:
# 4. a) How many movies with unique names are there? That is to say, count the distinct names of the movies.

import csv

movie_set = set()

with open('movie_titles.csv', encoding='iso-8859-1') as f:
  lines = csv.reader(f)
  for line in lines:
    if len(line) >= 2:
      movie = " ".join(line[2:]).strip()
    movie_set.add(movie)

print(len(movie_set))

17359


In [7]:
# 4. b) How many movie names refer to four different movies?

def refer_four_movies(filename):
  movie_names_count = {}
  res = 0

  with open(filename, encoding='iso-8859-1') as f:
    lines = csv.reader(f)

    for line in lines:
      if len(line) >= 2:
        movie_name = " ".join(line[2:]).strip()
        if movie_name in movie_names_count:
          movie_names_count[movie_name] += 1
        else:
          movie_names_count[movie_name] = 1

    for movie, count in movie_names_count.items():
      if count == 4:
        res += 1

  return res

refer_four_movies('movie_titles.csv')

5

In [ ]:
# 5. a) How many users rated exactly 200 movies?

combined_data_list = ['combined_data_1.txt', 'combined_data_2.txt', 'combined_data_3.txt', 'combined_data_4.txt']

def count_exact_movies(files):
  user_count = {}

  for txt in combined_data_list:
    with open(txt, "r") as f:
      for line in f:
        if len(line.split(",")) == 1:
          continue
        customer_id = int(line.split(",")[0])
        if customer_id in user_count:
          user_count[customer_id] += 1
        else:
          user_count[customer_id] = 1

  return user_count

user_count = count_exact_movies(combined_data_list)

print(sum(v == 200 for v in user_count.values()))

605


In [ ]:
# 5. b) of these users, take the lowest user ID and print out the names of the movies that this person liked the most (all 5 star ratings).

def find_lowest_user_id(user_count):
  users = set()
  for id, count in user_count.items():
    if count == 200:
      users.add(id)
  return min(users)

find_lowest_user_id(user_count)

508

In [ ]:
from collections import defaultdict

def find_movie_id_list(user_id):
  customer_id_movies = defaultdict(set)

  for i in range(1, 5):
    with open(f'combined_data_{i}.txt', "r") as f:
      lines = f.readlines()
    for line in lines:
      line = line.rstrip()
      if line.endswith(":"):
        movie_id = int(line.split(":")[0])
      row_data = line.split(",")
      if len(row_data) == 3:
        customer_id = int(row_data[0])
        # rating
        if int(row_data[1]) == 5:
          customer_id_movies[customer_id].add(movie_id)
  return customer_id_movies[user_id]

In [ ]:
lowest_user_id = find_lowest_user_id(user_count)
movie_id_list = find_movie_id_list(lowest_user_id)
print(movie_id_list)

{11521, 11907, 14601, 6029, 8846, 7057, 2452, 10774, 14358, 14999, 4633, 17174, 13856, 2465, 5538, 12195, 14240, 2342, 3371, 2862, 14382, 12084, 12852, 2743, 312, 571, 6974, 7230, 15296, 8387, 12870, 8904, 1865, 2122, 7755, 12232, 3917, 8782, 14671, 720, 3282, 6099, 12500, 14550, 9818, 16604, 17632, 5601, 17764, 5862, 15339, 494, 2158, 14961, 5237, 4727, 4345}


In [ ]:
import pandas as pd

df = pd.read_csv("/content/movie_titles.csv", encoding="latin1", usecols=[1, 2], header=None, names=['year', 'movie']).shift()[1:]
df.head(10)
for movie_id in movie_id_list:
  print(df['movie'][movie_id])

Lord of the Rings: The Two Towers
Cabaret
Harold and Maude
Amelie
Election
Lord of the Rings: The Two Towers: Extended Edition
Lord of the Rings: The Fellowship of the Ring
Taxi Driver
Monty Python and the Holy Grail
Monster
Gandhi
Maria Full of Grace
Raging Bull
This Is Spinal Tap
Monty Python's Life of Brian
The Accused
Lord of the Rings: The Return of the King
Super Size Me
Whale Rider
The Silence of the Lambs
Raising Arizona
Adaptation
To Be and To Have
The Pianist
High Fidelity
American Beauty
The Usual Suspects
The Lord of the Rings: The Fellowship of the Ring: Extended Edition
Band of Brothers
Minority Report
Schindler's List
Good Will Hunting
Eternal Sunshine of the Spotless Mind
Being John Malkovich
Touching the Void
Lost in Translation
Garden State
The Royal Tenenbaums
Downfall
Roger & Me
Sideways
Apocalypse Now
Boys Don't Cry
The Shawshank Redemption: Special Edition
L.A. Confidential
Unforgiven
Days of Wine and Roses
The Manchurian Candidate
Shakespeare in Love
Memento
Thre